In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import os
from konlpy.tag import Komoran
from tqdm.notebook import tqdm
import emoji
from konlpy.tag import Okt
import ast
from collections import Counter, defaultdict
import wordcloud
from wordcloud import WordCloud
from matplotlib import pyplot as plt
from matplotlib import font_manager
import seaborn as sns
import glob
import re

# 데이터 읽기

In [2]:
opath ="D:/멀티캠퍼스/비타민_프로젝트/preprocess/final_token/png3/"
path = "D:/멀티캠퍼스/비타민_프로젝트/final_data/"
file_list = glob.glob(path + "*.csv")  # .csv 파일만 가져옴

# 파일명만 추출
file_list = [file.split("\\")[-1] for file in file_list]
file_list

['01등_01고려은단_reviews.csv',
 '02등_02오쏘몰_reviews.csv',
 '03등_03네추럴라이즈_reviews.csv',
 '04등_04센트룸_reviews.csv',
 '05등_05뉴트리코어_reviews.csv',
 '06등_06덴프스_reviews.csv',
 '07등_07닥터프로그램_reviews.csv',
 '08등_08네츄럴라이프얼라이브_reviews.csv',
 '09등_09바이엘_reviews.csv',
 '10등_10인테로_reviews.csv',
 '11등_11바이엘_reviews.csv',
 '12등_12뉴트리코어_reviews.csv',
 '13등_13아임비타_reviews.csv',
 '14등_14함소아_reviews.csv',
 '15등_15GNM자연의품격_reviews.csv',
 '16등_16네츄럴라이트얼라이브_reviews.csv',
 '17등_17GNM자연의품격_reviews.csv',
 '18등_18센트룸_reviews.csv',
 '19등_19세노비스_reviews.csv',
 '20등_20유한양행_reviews.csv']

In [5]:
file_name = file_list[0]

pattern = r'(\d{2})등_(\d{2})(.*?)_reviews\.csv'
match = re.search(pattern, file_name)
if match:
    top = match.group(2)
    name = match.group(3)
    print(f'top: {top}')
    print(f'name: {name}')

top: 01
name: 고려은단


In [7]:
df = pd.read_csv(path + file_name, on_bad_lines="skip")
df[["year","month", "day", "no"]] = df["date"].str.split(".", expand=True)
df = df.drop("no", axis= 1)
df["year"] = "20" + df["year"]
df["date_full"] =  df["year"] + "-" + df["month"] + "-" + df["day"]
df["date_full"] = pd.to_datetime(df["date_full"], format="%Y-%m-%d")
df
# df

,top,url,brand,page,vitamin_title,date,sex,score,review,year,month,day,date_full
0,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.08.07.,NaN,5,알이 크다고 해서 걱정했는데 생각보다 잘 넘겨지더라고요. 괜히 돈 더내고 작은거 살...,2024,08,07,2024-08-07
1,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.08.07.,여성,5,원래 고려은단 비타민 D이지\n먹고있었는데 칼슘 마그네슘 들어간것이 필요해 ...,2024,08,07,2024-08-07
2,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.08.10.,NaN,5,몇년째 꾸준히 섭취하고있는 비타민이예요 하루한알만 챙기면 든든하네요 성분 함량 모두...,2024,08,10,2024-08-10
3,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.08.10.,남성,5,고려은단이라면 믿을수 있을수 있다고 봅니다 \n회사가 생긴지 좀 됐고 제품을 만든것...,2024,08,10,2024-08-10
4,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.08.08.,NaN,5,빠른 배송과저렴한 가격 고품질 제품 매우만족합니다.\n재구매 제품입니다.\n추천합니...,2024,08,08,2024-08-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1000,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.04.13.,NaN,5,4인가족이 함께 먹어요~\n가격대비 만족합니다,2024,04,13,2024-04-13
19996,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1000,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.04.22.,NaN,4,먹어보고 좋아서 또 구매했어요ㆍ,2024,04,22,2024-04-22
19997,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1000,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.05.03.,남성,5,항상 복용하는제품입니다. 만족하고 있습니다.,2024,05,03,2024-05-03
19998,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1000,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.04.02.,NaN,5,두 달 정도 먹었는데 확실히 피로감이 덜 하고 감기 걸려도 금방 회복되요 생활이 편...,2024,04,02,2024-04-02


#### 이모지 없애기

In [8]:
def remove_emoji(df):

    for i in tqdm(df.index):
      df.loc[i, 'review'] = emoji.replace_emoji(df.loc[i, 'review'], replace='')
            
    return df

In [9]:
remove_emoji(df)

  0%|          | 0/20000 [00:00<?, ?it/s]

,top,url,brand,page,vitamin_title,date,sex,score,review,year,month,day,date_full
0,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.08.07.,NaN,5,알이 크다고 해서 걱정했는데 생각보다 잘 넘겨지더라고요. 괜히 돈 더내고 작은거 살...,2024,08,07,2024-08-07
1,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.08.07.,여성,5,원래 고려은단 비타민 D이지\n먹고있었는데 칼슘 마그네슘 들어간것이 필요해 ...,2024,08,07,2024-08-07
2,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.08.10.,NaN,5,몇년째 꾸준히 섭취하고있는 비타민이예요 하루한알만 챙기면 든든하네요 성분 함량 모두...,2024,08,10,2024-08-10
3,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.08.10.,남성,5,고려은단이라면 믿을수 있을수 있다고 봅니다 \n회사가 생긴지 좀 됐고 제품을 만든것...,2024,08,10,2024-08-10
4,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.08.08.,NaN,5,빠른 배송과저렴한 가격 고품질 제품 매우만족합니다.\n재구매 제품입니다.\n추천합니...,2024,08,08,2024-08-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1000,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.04.13.,NaN,5,4인가족이 함께 먹어요~\n가격대비 만족합니다,2024,04,13,2024-04-13
19996,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1000,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.04.22.,NaN,4,먹어보고 좋아서 또 구매했어요ㆍ,2024,04,22,2024-04-22
19997,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1000,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.05.03.,남성,5,항상 복용하는제품입니다. 만족하고 있습니다.,2024,05,03,2024-05-03
19998,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1000,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.04.02.,NaN,5,두 달 정도 먹었는데 확실히 피로감이 덜 하고 감기 걸려도 금방 회복되요 생활이 편...,2024,04,02,2024-04-02


# Token

In [10]:
# Komoran 객체 생성
komoran = Komoran(userdic="D:/멀티캠퍼스/비타민_프로젝트/user_1.dic")
# Okt 객체 생성
okt = Okt()

# Komoran을 사용하여 명사 추출
def extract_nouns(sentence):
    try:
        tokens = komoran.pos(sentence)
        nouns = [x[0] for x in tokens if x[1] in ("NNG", "NNP")]
        return nouns
    except Exception as e:
        print(e, sentence)
        return []

# Komoran을 사용하여 명사와 동사 추출

# Okt를 사용하여 명사와 동사 조합 및 명사만 추출
def extract_nouns_and_verbs(sentence):
    try:
        tokens = okt.pos(sentence)
        nouns_and_verbs = []
        current_noun = ""
        
        for word, pos in tokens:
            if pos == 'Noun':
                current_noun = word
            elif pos == 'Verb' and current_noun:
                nouns_and_verbs.append(current_noun + " " + word)
                current_noun = ""
        
        return nouns_and_verbs
    except Exception as e:
        print(e, sentence)
        return []

tqdm.pandas()

# 'nouns'와 'nouns_and_verbs' 컬럼 생성
def process_row(review):
    nouns = extract_nouns(review)
    nouns_and_verbs = extract_nouns_and_verbs(review)
    return pd.Series([nouns, nouns_and_verbs],
                     index=['nouns', 'nouns_and_verbs'])

df[['nouns', 'nouns_and_verbs']] = df['review'].progress_apply(process_row)
df

  0%|          | 0/20000 [00:00<?, ?it/s]

java.lang.NullPointerException: Cannot invoke "java.lang.Integer.intValue()" because the return value of "kr.co.shineware.util.common.model.Pair.getFirst()" is null 라이브 방송 때 주문했어요. 
고려은단 비타민 C를 워낙 오래 먹어와서 고려은단의 제품력에 대한 믿음이 있고 
이 제품 또한 예전에 먹어본 적이 있습니다. 
당시 만족하였으나 이런저런 이유로 그만 먹게 되었다가 
이번에 건강증진을 위해 다시 주문했어요.
 
먹을 때 역함이 전혀 없고
유통기한이 깁니다.
사이즈는 고려은단 비타민C와 같은 사이즈예요.
이번 구매는 가격 혜택도 좋았답니다.

잘 먹을게요. 번창하세요^^
<built-in method __subclasscheck__ of _jpype._JClass object at 0x000001FD4C5C57C0> returned a result with an exception set     


,top,url,brand,page,vitamin_title,date,sex,score,review,year,month,day,date_full,nouns,nouns_and_verbs
0,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.08.07.,NaN,5,알이 크다고 해서 걱정했는데 생각보다 잘 넘겨지더라고요. 괜히 돈 더내고 작은거 살...,2024,08,07,2024-08-07,"[알, 걱정, 생각, 돈, 종비, 하루만, 티, 주문, 비타민c, 시너지, 효과, ...","[알 크다고, 걱정 했는데, 생각 잘, 살뻔 했네요, 비만 먹는건, 티 날텐데, 비..."
1,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.08.07.,여성,5,원래 고려은단 비타민 D이지\n먹고있었는데 칼슘 마그네슘 들어간것이 필요해 ...,2024,08,07,2024-08-07,"[원래, 고려은단, 비타민 D, 칼슘, 마그네슘, 필요, 올인원, 구입, 올인원, ...","[이지 먹고있었는데, 마그네슘 들어간것이, 구입 했어요, 조금 큰건데, 주문 했어요..."
2,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.08.10.,NaN,5,몇년째 꾸준히 섭취하고있는 비타민이예요 하루한알만 챙기면 든든하네요 성분 함량 모두...,2024,08,10,2024-08-10,"[섭취, 비타민, 이예, 하루, 알, 성분, 함량, 마음]","[섭취 하고있는, 알 챙기면, 마음 들어요]"
3,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.08.10.,남성,5,고려은단이라면 믿을수 있을수 있다고 봅니다 \n회사가 생긴지 좀 됐고 제품을 만든것...,2024,08,10,2024-08-10,"[고려은단, 회사, 제품, 실력, 나이, 사람, 큰일, 없다]","[은 단이라면, 회사 생긴지, 좀 됐고, 제품 만든것도, 좀 됐고, 나이 어린, 큰..."
4,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.08.08.,NaN,5,빠른 배송과저렴한 가격 고품질 제품 매우만족합니다.\n재구매 제품입니다.\n추천합니...,2024,08,08,2024-08-08,"[배송, 가격, 품질, 제품, 매우, 만족, 재구매, 제품, 추천, 감사, 사업, 번창]","[매우 재구매, 추천 합니다, 번창 하십시요]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1000,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.04.13.,NaN,5,4인가족이 함께 먹어요~\n가격대비 만족합니다,2024,04,13,2024-04-13,"[가족, 가격, 대비, 만족]",[가족 먹어요]
19996,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1000,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.04.22.,NaN,4,먹어보고 좋아서 또 구매했어요ㆍ,2024,04,22,2024-04-22,"[좋아서, 구매]",[구매 했어요]
19997,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1000,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.05.03.,남성,5,항상 복용하는제품입니다. 만족하고 있습니다.,2024,05,03,2024-05-03,"[복용, 제품, 만족]",[복용 하는]
19998,1,https://brand.naver.com/koreaeundanhc/products...,고려은단,1000,"고려은단 멀티비타민 올인원 60캡슐, 3개",24.04.02.,NaN,5,두 달 정도 먹었는데 확실히 피로감이 덜 하고 감기 걸려도 금방 회복되요 생활이 편...,2024,04,02,2024-04-02,"[달, 정도, 피로감, 감기, 회복, 생활, 가격, 할인, 행사]","[정도 먹었는데, 덜 하고, 감기 걸려도, 회복 되요, 행사 하면]"


In [25]:
TOKEN = df.to_csv(opath + f'{top}_{name}_df_token.csv', encoding="utf-8-sig", index=False)